<div style = 'border : 3px solid lightblue; background-color:#ABB9AB;padding:10px'>

* **[1.專案的目的 📜](#1)**

  - 對專案想法的簡單定義
    

* **[2.導入庫 📚](#2)**  

   - 回顧完成專案的最重要庫 
    

* **[3.探索性數據分析 (EDA) 📊](#3)**  

    - 通過數據及其在圖表中的表示了解信息

    - 知道數據正常形式中不清晰的部分
      

* **[4.數據預處理 🔧](#4)**  
    
   - 數據切割 
    
   - 數據生成器 
    

* **[5.加載預訓練模型](#5)**  
    
    - 什麼是遷移學習
    
    - 回調函數
    
    - 訓練模型
    
    - 繪製模型
    
   
* **[6.模型評估 📈](#6)**  
    
    - 分類報告
    
    - 混淆矩陣
    
   
* **[7.Grad-Cam](#7)**
    
    - 簡單說明 CNN 如何看待


<center>
<img src="https://i.pinimg.com/originals/cd/5f/c4/cd5fc4478e62ab5ca0bfe091fdb58a50.gif" alt="error" width="400" height="400"></center>

<a id="1"></a>
# <p style="padding:10px;background-color:#8DA48E ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">專案的目的 📜</p>

<div style = 'border : 3px solid lightblue; background-color:#ABB9AB;padding:10px'>

🔘 **問題**：魚類被認為是全球人口最重要的食物來源之一。隨著人口的增長和魚類消費的增加，對魚類的分類、識別和包裝需要花費大量的時間以及人力和物力成本。然而，隨著現代機械的出現，這一過程可以更輕鬆地完成，同時大幅降低成本。我們還可以保證持續運作，這意味著可以全天候工作，從而實現更好的魚類生產、分類和包裝。
    
🔘 **解決方案**：可以訓練一個模型來識別某些魚類，然後通過卷積神經網絡進行分類，並根據需要在許多與魚類相關的領域中使用它。


<a id="2"></a>
# <p style="padding:10px;background-color:#8DA48E ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">導入庫 📚</p>


In [ ]:
import os #導入操作系統庫
import glob as gb #導入文件操作庫
import numpy as np #導入數據處理庫
import pandas as pd #導入數據處理庫
import seaborn as sns #導入視覺化庫
from pathlib import Path #導入路徑庫
import matplotlib.cm as cm #導入colormap
import matplotlib.pyplot as plt #導入繪圖庫
plt.style.use('Solarize_Light2') #設置繪圖風格

import warnings
warnings.filterwarnings('ignore') #忽略警告

import tensorflow as tf #導入tensorflow
from tensorflow.keras.models import Sequential,Model #導入模型
from tensorflow.keras.applications import MobileNetV2,VGG16 #導入預訓練模型
from tensorflow.keras.layers import Conv2D,Flatten,Dropout,BatchNormalization,Dense #導入層
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint #導入回調函數
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,array_to_img,img_to_array #導入圖像生成器

from sklearn.model_selection import train_test_split #導入數據集劃分
from sklearn.metrics import confusion_matrix,classification_report #導入混淆矩陣和分類報告

<a id="3"></a>
# <p style="padding:10px;background-color:#8DA48E ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">探索性數據分析 (EDA) 📊 </p>


**<p style="color:#6D4318">準備所有數據路徑</p>**

In [ ]:
fish_path=r"D:\Learning_Python\Exercises_Machine_Learning\A Large Scale Fish Dataset\archive\Fish_Dataset\Fish_Dataset"

**<p style="color:#6D4318">找出每個資料夾中的圖像數量</p>**


In [ ]:
all_path = []  # 包含每個圖像的完整路徑
for img_path in os.listdir(fish_path): 
    if img_path in ['Segmentation_example_script.m', 'README.txt', 'license.txt']:
        continue  # 如果是這些特定的檔案，則跳過

    all_data = gb.glob(pathname=fish_path + '/' + img_path + '/' + img_path + '/*.*')
    print(' 在 {} 中找到 {} '.format(len(all_data), img_path))
    all_path.extend(all_data)  # 將每個圖像的完整路徑加入到之前提到的 all_path 列表中

**<p style="color:#6D4318">創建圖像資料框</p>**

In [ ]:
# 創建圖像的資料框
images_df = pd.DataFrame({'Filepath': all_path})
images_df['Label'] = images_df['Filepath'].apply(lambda x: x.split('/')[-2])  # 從路徑中提取標籤
pd.options.display.max_colwidth = 200  # 設置顯示的最大列寬

# 隨機打亂資料並重置索引
images_df = images_df.sample(frac=1).reset_index(drop=True)
images_df.head(5)  # 顯示前5行

**<p style="color:#6D4318">找出數據中每個標題的分佈情況</p>**

In [ ]:
# 設定圖形大小為 15x5
plt.figure(figsize=(15, 5))

# 繪製每個標籤的計數條形圖
plt.subplot(1, 2, 1)
sns.countplot(data=images_df, x='Label')
plt.xticks(rotation=60)

# 繪製每個標籤的分佈餅圖
plt.subplot(1, 2, 2)
plt.pie(x=images_df['Label'].value_counts().values, 
        labels=images_df['Label'].value_counts().index, 
        autopct='%1.1f%%')

# 設置整體標題
plt.suptitle('數據中每個類別的分佈', size=20)
plt.show()  # 顯示圖形

<div style = 'border : 3px solid lightblue; background-color:#ABB9AB;padding:10px'>

**<p style="color:red">觀察結果 📋</p>**    

🔘 數據是平衡的，每個類別包含相等數量的圖像 **(1000)** 張圖像

**<p style="color:#6D4318">顯示數據集中20張圖片</p>**

In [ ]:
# 創建一個 4x5 的圖形網格，大小為 15x7
fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(15, 7),
                        subplot_kw={'xticks': [], 'yticks': []})  # 不顯示 x 和 y 軸刻度

# 迴圈顯示圖片並設置標題為對應的標籤
for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(images_df.Filepath[i]))  # 顯示圖片
    ax.set_title(images_df.Label[i])  # 設置標題為標籤

plt.tight_layout()  # 自動調整子圖間的間距
plt.show()  # 顯示圖形

**<p style="color:#6D4318">數據切割</p>**

In [ ]:
# 將數據集切分為訓練集和測試集，測試集佔總數的 10%
training_df, testing_df = train_test_split(images_df, test_size=0.1, shuffle=True, random_state=1)

# 打印訓練數據和測試數據的維度
print('訓練數據的維度:', training_df.shape)
print('測試數據的維度:', testing_df.shape)

**<p style="color:#6D4318">數據生成器</p>**

In [ ]:
# 創建訓練數據生成器，使用 VGG16 的預處理函數，並進行 20% 的驗證數據劃分
training_generator = ImageDataGenerator(
    tf.keras.applications.vgg16.preprocess_input,
    validation_split=0.2,
)

# 創建測試數據生成器，使用 VGG16 的預處理函數
testing_generator = ImageDataGenerator(
    tf.keras.applications.vgg16.preprocess_input
)

In [ ]:
# 創建訓練數據生成器
training_images = training_generator.flow_from_dataframe(
    dataframe=training_df,  # 使用訓練資料框
    x_col='Filepath',  # 圖像路徑
    y_col='Label',  # 圖像標籤
    class_mode='categorical',  # 多類別分類
    target_size=(224, 224),  # 圖像大小調整為 224x224
    color_mode='rgb',  # 使用 RGB 色彩模式
    batch_size=32,  # 批次大小為 32
    shuffle=True,  # 隨機打亂數據
    seed=42,  # 隨機種子
    subset='training'  # 用於訓練子集
)

# 創建驗證數據生成器
validation_images = training_generator.flow_from_dataframe(
    dataframe=training_df,  # 使用訓練資料框
    x_col='Filepath',  # 圖像路徑
    y_col='Label',  # 圖像標籤
    class_mode='categorical',  # 多類別分類
    target_size=(224, 224),  # 圖像大小調整為 224x224
    color_mode='rgb',  # 使用 RGB 色彩模式
    batch_size=32,  # 批次大小為 32
    shuffle=True,  # 隨機打亂數據
    seed=42,  # 隨機種子
    subset='validation'  # 用於驗證子集
)

# 創建測試數據生成器
testing_images = testing_generator.flow_from_dataframe(
    dataframe=testing_df,  # 使用測試資料框
    x_col='Filepath',  # 圖像路徑
    y_col='Label',  # 圖像標籤
    class_mode='categorical',  # 多類別分類
    target_size=(224, 224),  # 圖像大小調整為 224x224
    color_mode='rgb',  # 使用 RGB 色彩模式
    batch_size=32,  # 批次大小為 32
    shuffle=False  # 不打亂數據（測試集通常不打亂）
)

<a id="5"></a>
# <p style="padding:10px;background-color:#8DA48E ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">加載預訓練模型</p>

<div style = 'border : 3px solid lightblue; background-color:#ABB9AB;padding:10px'>

**什麼是遷移學習**
    
🔘 遷移學習在卷積神經網絡 (CNN) 中是指使用在相似任務上已經訓練過的模型作為訓練新模型的起點，應用於不同的任務。

**遷移學習的好處** 
    
🔘 主要優勢是節省訓練時間，神經網絡在大多數情況下表現更好，而且不需要大量數據。

<div style = 'border : 3px solid lightblue; background-color:#ABB9AB;padding:10px'>

🔘 **VGG-16**
   是一個具有 16 層深度的卷積神經網絡。您可以加載在 ImageNet 數據庫中，基於超過一百萬張圖片訓練過的預訓練版本。

🔘 預訓練的網絡可以將圖像分類為 1000 個物體類別，例如鍵盤、滑鼠、鉛筆和許多動物。

<center>
<img src="https://miro.medium.com/v2/resize:fit:827/1*UeAhoKM0kJfCPA03wt5H0A.png" alt="error" width="700" height="600"></center>

In [ ]:
# 加載 VGG16 預訓練模型
pretrained_model = VGG16(
    input_shape=(224, 224, 3),  # 設置輸入圖像的形狀為 224x224，並具有 3 個通道 (RGB)
    include_top=False,  # 不包含頂部的全連接層
    weights='imagenet',  # 使用在 ImageNet 數據集上訓練的權重
    pooling='avg'  # 使用全局平均池化來替代全連接層
)

# 凍結預訓練模型的權重，防止在訓練過程中被更新
pretrained_model.trainable = False

**<p style="color:#6D4318">回調函數</p>**

<div style = 'border : 3px solid lightblue; background-color:#ABB9AB;padding:10px'>

🔘 **早停法 (Early Stopping)** 是深層神經網絡中的一種正則化技術，當參數更新不再對驗證集的結果帶來改進時，訓練將停止。

🔘 **學習率衰減 (ReduceLROnPlateau)** 會在某個指標不再改進時減小學習率。這個回調函數會監控一個指標，如果在 '耐心' 設定的回合數內未看到改善，學習率就會被降低。

🔘 **模型檢查點 (ModelCheckpoint)** 是用來定期保存 Keras 模型或模型權重的回調函數。

In [ ]:
# 定義早停回調函數
early_stopping = EarlyStopping(
    monitor='val_loss',  # 監控驗證集的損失
    patience=3,  # 如果 3 個回合內驗證損失沒有改善則停止訓練
    verbose=1,  # 顯示訓練過程的詳細信息
    restore_best_weights=True  # 停止訓練時恢復最佳權重
)

# 定義學習率衰減回調函數
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # 監控驗證集的損失
    patience=2,  # 如果 2 個回合內損失未改善，則減小學習率
    verbose=0,  # 不顯示詳細信息
    factor=0.1  # 當條件滿足時，學習率縮小為原來的 0.1 倍
)

# 定義模型檢查點回調函數
model_check_point = ModelCheckpoint(
    monitor='val_accuracy',  # 監控驗證集的準確率
    filepath='./bestmodel.h5',  # 保存最佳模型的文件路徑
    save_best_only=True,  # 只保存最好的模型
    verbose=True  # 顯示保存過程的詳細信息
)

**<p style="color:#6D4318">訓練模型 </p>**

In [ ]:
# 設置訓練的回合數和批次大小
epochs = 7
batch_size = 32

In [ ]:
# 定義模型結構
inputs = pretrained_model.input  # 使用預訓練模型的輸入層

x = Dense(128, activation='relu')(pretrained_model.output)  # 全連接層，128 個神經元，激活函數為 ReLU
x = Dense(128, activation='relu')(x)  # 再添加一個 128 個神經元的全連接層
x = Dropout(rate=0.3)(x)  # 隨機丟棄 30% 的神經元來防止過擬合
x = BatchNormalization()(x)  # 批次正規化以加速訓練
x = Dense(64, activation='relu')(x)  # 添加 64 個神經元的全連接層

outputs = Dense(9, activation='softmax')(x)  # 輸出層，9 個分類，激活函數為 softmax

# 創建模型
model = Model(inputs=inputs, outputs=outputs)

# 編譯模型，設置優化器、損失函數和評估指標
model.compile(
    optimizer='adam',  # 使用 Adam 優化器
    loss='categorical_crossentropy',  # 多類別交叉熵損失函數
    metrics=['accuracy']  # 使用準確率作為評估指標
)

# 訓練模型
history = model.fit(
    training_images,  # 訓練數據
    validation_data=validation_images,  # 驗證數據
    epochs=epochs,  # 訓練回合數
    batch_size=batch_size,  # 每個批次的數據量
    callbacks=[early_stopping, reduce_lr, model_check_point]  # 使用早停、學習率衰減和模型檢查點回調函數
)

**<p style="color:#6D4318">模型摘要</p>**

In [ ]:
# 顯示模型摘要（模型結構）
model.summary()

**<p style="color:#6D4318">繪製模型圖</p>**

In [ ]:
# 繪製模型結構圖
tf.keras.utils.plot_model(model)

**<p style="color:#6D4318">模型歷史資料框</p>**

In [ ]:
# 將訓練歷史記錄轉換為資料框
history_df = pd.DataFrame(history.history)
history_df  # 顯示資料框

In [ ]:
# 設置圖形大小為 15x5
plt.figure(figsize=(15, 5))

# 繪製準確率和驗證準確率的變化圖
plt.subplot(1, 2, 1)
plt.plot(history_df['accuracy'], label='accuracy', c='red')  # 繪製訓練準確率
plt.plot(history_df['val_accuracy'], label='val_accuracy')  # 繪製驗證準確率
plt.xlabel('回合數')  # 設置 x 軸標籤為回合數
plt.ylabel('準確率')  # 設置 y 軸標籤為準確率
plt.title('訓練準確率 VS 驗證準確率')  # 設置圖標題
plt.legend()  # 顯示圖例

# 繪製損失值和驗證損失值的變化圖
plt.subplot(1, 2, 2)
plt.plot(history_df['loss'], label='loss', c='red')  # 繪製訓練損失值
plt.plot(history_df['val_loss'], label='val_loss')  # 繪製驗證損失值
plt.title('訓練損失值 VS 驗證損失值')  # 設置圖標題
plt.xlabel('回合數')  # 設置 x 軸標籤為回合數
plt.ylabel('損失值')  # 設置 y 軸標籤為損失值
plt.legend()  # 顯示圖例

# 顯示所有繪製的圖
plt.show()

**<p style="color:#6D4318">加載驗證準確率最高的模型</p>**

In [ ]:
# 加載最佳驗證準確率的模型
from keras.models import load_model

model = load_model('/kaggle/working/bestmodel.h5')  # 加載保存的最佳模型

**<p style="color:#6D4318">在測試數據上評估模型</p>**

In [ ]:
# 在測試數據上評估模型
Evaluation = model.evaluate(testing_images)

# 輸出測試準確率和損失值
print("測試準確率: {:.2f}%".format(Evaluation[1] * 100))
print("測試損失值: {:.5f}".format(Evaluation[0]))

**<p style="color:#6D4318">模型預測</p>**

In [ ]:
# 對測試數據進行預測
prediction = model.predict(testing_images)
# 使用 argmax 找出每個預測結果中的最大值索引（即預測的類別）
prediction = np.argmax(prediction, axis=1)

In [ ]:
# 查看訓練數據中各類別的索引對應關係
training_images.class_indices

In [ ]:
# 取得類別索引對應的標籤
labels = (training_images.class_indices)
# 將類別索引和標籤對調，以便從預測結果中還原標籤
labels = dict((v, k) for k, v in labels.items())
# 根據預測結果的索引還原為標籤
prediction = [labels[k] for k in prediction]

<a id="6"></a>
# <p style="padding:10px;background-color:#8DA48E ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">模型評估 📈 </p>

In [ ]:
# 獲取測試數據中的真實標籤
y_true = testing_df['Label'].values
# 輸出分類報告，顯示模型的準確率、召回率和 F1 值等
print(classification_report(y_true, prediction))

In [ ]:
# 計算混淆矩陣
cm = confusion_matrix(y_true, prediction)
# 設置圖形大小
plt.figure(figsize=(15, 5))
# 繪製混淆矩陣的熱圖
sns.heatmap(cm, annot=True, cmap='Blues', linewidths=0.5)
# 設置 x 軸和 y 軸標籤
plt.xlabel('預測標籤')  # 'predicted label'
plt.ylabel('真實標籤')  # 'True label'
# 設置圖標題
plt.title('9 類別的混淆矩陣')
# 顯示圖形
plt.show()

**<p style="color:#6D4318">準備預測值的資料框</p>**

In [ ]:
# 創建一個複製的測試數據框並添加預測結果
temp_df = testing_df.copy()
temp_df['predicted'] = prediction  # 添加預測的標籤列

# 將實際標籤與預測標籤進行比較，並標記是否相同
temp_df.loc[temp_df['Label'] == temp_df['predicted'], 'Same'] = 'True'
temp_df.loc[temp_df['Label'] != temp_df['predicted'], 'Same'] = 'False'

# 重置索引
temp_df = temp_df.reset_index(drop=True)
temp_df.head()  # 顯示前 5 行

In [ ]:
# 設置圖形大小
plt.figure(figsize=(15, 5))

# 繪製預測結果是否正確的柱狀圖
plt.subplot(1, 2, 1)
sns.countplot(data=temp_df, x='Same')

# 繪製預測結果是否正確的餅圖
plt.subplot(1, 2, 2)
plt.pie(x=temp_df['Same'].value_counts().values, 
        labels=temp_df['Same'].value_counts().index,
        autopct='%1.1f%%', 
        explode=[0.1, 0.1])

# 設置圖表的總標題
plt.suptitle('預測圖像的模型評估', size=20)
plt.show()  # 顯示圖表

In [ ]:
# 定義函數來顯示圖像
def display_image(temp_df):
    '''
    輸入 : 數據框
    
    輸出 : 顯示數據框中的 8 張圖片
    '''
    # 創建一個 2x4 的圖形網格，大小為 15x7
    fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15, 7),
                             subplot_kw={'xticks': [], 'yticks': []})

    # 顯示圖片及其實際標籤和預測標籤
    for i, ax in enumerate(axes.flat):
        ax.imshow(plt.imread(temp_df.Filepath.iloc[i]))
        ax.set_title(f"實際: {temp_df.Label.iloc[i]}\n預測: {temp_df.predicted.iloc[i]}") 
    plt.tight_layout()  # 自動調整子圖之間的間距
    plt.show()  # 顯示圖像

**<p style="color:#6D4318">顯示預測正確的圖像</p>**

In [ ]:
# 顯示預測正確的圖像
display_image(temp_df[temp_df['Same'] == 'True'])

**<p style="color:#6D4318">顯示預測錯誤的圖像</p>**

In [ ]:
# 顯示預測錯誤的圖像
display_image(temp_df[temp_df['Same'] == 'False'])

<a id="7"></a>
# <p style="padding:10px;background-color:#8DA48E ;margin:0;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 15px 50px;overflow:hidden;font-weight:500">Grad-Cam </p>

<div style = 'border : 3px solid lightblue; background-color:#ABB9AB;padding:10px'>

🔘 返回梯度加權類別激活映射 **(Grad-CAM)**

🔘 使用流入 CNN 最後一層卷積層的梯度信息來理解每個神經元對所關注決策的貢獻。

🔘 **Grad-CAM** 方法用於提取深層神經網絡的特徵圖，隨後使用注意力機制來提取高層次的注意力圖。注意力圖突出顯示了圖像中對目標類別重要的區域，可以視為深層神經網絡的視覺解釋。

In [ ]:
# last_conv_layer='block5_conv3' # 您可以通過模型摘要來得知這一點

In [ ]:
import matplotlib.cm as cm

# 定義函數來獲取圖像的數組格式
def get_img_array(img_path, size):
    img = load_img(img_path, target_size=size)  # 加載圖像並調整大小
    array = img_to_array(img)  # 將圖像轉換為數組
    # 增加一個維度，將數組轉換為 "批次" 格式
    array = np.expand_dims(array, axis=0)
    return array

# 定義函數來生成 Grad-CAM 熱圖
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # 創建一個模型，將輸入圖像映射到最後一層卷積層的激活值以及模型的預測輸出
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # 計算輸入圖像的預測類別對應的梯度，相對於最後一層卷積層的激活值
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])  # 如果沒有指定預測類別，則使用預測概率最大的類別
        class_channel = preds[:, pred_index]

    # 計算輸出神經元（預測的類別）相對於最後一層卷積層輸出特徵圖的梯度
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # 將每個特徵圖通道的梯度取平均，得到一個向量
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # 將梯度加權應用到最後一層卷積層的輸出，生成熱圖
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # 將熱圖的值規範化到 0 到 1 之間，便於視覺化
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# 定義函數來保存並顯示 Grad-CAM 熱圖
def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # 加載原始圖像
    img = tf.keras.preprocessing.image.load_img(img_path)
    img = tf.keras.preprocessing.image.img_to_array(img)

    # 將熱圖重新調整到 0-255 的範圍
    heatmap = np.uint8(255 * heatmap)

    # 使用 jet 色彩映射來將熱圖進行著色
    jet = cm.get_cmap("jet")

    # 提取 RGB 色彩值
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # 創建一個帶有 RGB 色彩熱圖的圖像
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # 將熱圖疊加到原始圖像上
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = array_to_img(superimposed_img)

    # 保存疊加後的圖像
    superimposed_img.save(cam_path)

    # 顯示 Grad-CAM
#     display(Image(cam_path))
    
    return cam_path

# 預處理和解碼 VGG16 預測結果的函數
preprocess_input = tf.keras.applications.vgg16.preprocess_input
decode_predictions = tf.keras.applications.vgg16.decode_predictions

# 最後一層卷積層的名稱，來自模型摘要
last_conv_layer_name = "block5_conv3"
img_size = (224, 224)

# 移除最後一層的 softmax 激活函數
model.layers[-1].activation = None

In [ ]:
def display_heatmap_image(df):
    '''
    輸入 : 數據框
    
    輸出 : 顯示數據框中的 8 張 Grad-CAM 圖像
    '''
    
    # 創建一個 2x4 的圖形網格，大小為 15x10
    fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15, 10),
                             subplot_kw={'xticks': [], 'yticks': []})

    # 遍歷每個圖像，生成 Grad-CAM 熱圖並顯示
    for i, ax in enumerate(axes.flat):
        img_path = df.Filepath.iloc[i]  # 獲取圖像路徑
        img_array = preprocess_input(get_img_array(img_path, size=img_size))  # 預處理圖像數據
        heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)  # 生成 Grad-CAM 熱圖
        cam_path = save_and_display_gradcam(img_path, heatmap)  # 保存並顯示 Grad-CAM 圖像
        ax.imshow(plt.imread(cam_path))  # 顯示 Grad-CAM 圖像
        ax.set_title(f"實際: {df.Label.iloc[i]}\n預測: {df.predicted.iloc[i]}")  # 設置圖像標題為實際與預測標籤
    plt.tight_layout()  # 自動調整子圖間距
    plt.show()  # 顯示圖像

**<p style="color:#6D4318">顯示預測正確圖像的熱圖</p>**

In [ ]:
# 顯示預測正確圖像的 Grad-CAM 熱圖
display_heatmap_image(temp_df[temp_df['Same'] == 'True'])

<div style = 'border : 3px solid lightblue; background-color:#ABB9AB;padding:10px'>

**<p style="color:red">觀察結果 📋</p>**    

🔘 卷積神經網絡確定了每種魚類最重要的區分特徵

🔘 陰影部分顯示了卷積神經網絡進行分類決策的依據

**<p style="color:#6D4318">顯示預測錯誤圖像的熱圖</p>**

In [ ]:
# 顯示預測錯誤圖像的 Grad-CAM 熱圖
display_heatmap_image(temp_df[temp_df['Same'] == 'False'])

<div style = 'border : 3px solid lightblue; background-color:#ABB9AB;padding:10px'>

**<p style="color:red">觀察結果 📋</p>**    

🔘 卷積神經網絡無法清晰地識別部分區域，這可能是由於兩種不同魚類的大小或顏色相似，再加上圖像模糊所致。

<div style = 'border : 3px solid lightblue; background-color:#ABB9AB;padding:10px'>

# [感謝觀看到最後。如果您從這個筆記本中受益，請支持我點贊 ❤️](https://www.kaggle.com/code/mohamedwasef/acc-98-fish-classification-with-grad-cam/comments)